In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#import keras as keras
plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

## Load training data

In [ ]:
nb_classes = 10

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

Let's look at some examples of the training data

In [ ]:
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[i]))

In [ ]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train,
          batch_size=128, 
          epochs=4,
          #show_accuracy=True, 
          verbose=1,
          validation_data=(X_test, Y_test)
          )

## Finally, evaluate its performance

In [ ]:
score = model.evaluate(X_test, Y_test,
                       #show_accuracy=True, 
                       verbose=0)
print(score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
#predicted_classes = model.predict_classes(X_test)
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)


# Check which items we got right / wrong
correct_indices = np.nonzero(classes_x == y_test)[0]
incorrect_indices = np.nonzero(classes_x != y_test)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(classes_x[correct], y_test[correct]))
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(classes_x[incorrect], y_test[incorrect]))

In [ ]:
import cv2
from PIL import Image, ImageDraw
import numpy as np

# Função para preprocessar a imagem usando a biblioteca Pillow
def preprocess_image(window):
    # Converte a imagem para escala de cinza
    gray = window.convert('L')
    # Redimensiona a imagem para 28x28 pixels
    resized = gray.resize((28, 28))
    # Converte a imagem para um array numpy
    image_array = np.array(resized)
    # Inverte a cor da imagem (pixels brancos para pretos e vice-versa)
    inverted = np.invert(image_array)
    # Normaliza os valores dos pixels para o intervalo [0, 1]
    normalized = inverted / 255.0
    # Redimensiona a imagem para a entrada do modelo (1, 784)
    flattened = normalized.reshape(1, 784)
    return flattened

# Função para exibir a previsão da rede neural
def display_prediction(prediction):
    label = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    print(f"Prediction: {label}\nConfidence: {confidence:.2f}%")

# Código principal

# Crie uma nova janela usando o OpenCV
window = np.ones((280, 280), dtype=np.uint8) * 255
cv2.namedWindow('Draw a Digit')

# Variáveis para controle do desenho
drawing = False
last_point = (0, 0)

# Callback de eventos do mouse para desenhar na janela
def mouse_callback(event, x, y, flags, param):
    global drawing, last_point
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_point = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.line(window, last_point, (x, y), color=0, thickness=20)
            last_point = (x, y)

# Registre o callback de eventos do mouse
cv2.setMouseCallback('Draw a Digit', mouse_callback)

# Loop principal
while True:
    # Exiba a janela
    cv2.imshow('Draw a Digit', window)

    # Verifique os eventos do teclado
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('r'):
        window = np.ones((280, 280), dtype=np.uint8) * 255
    elif key == ord('c'):
        image = preprocess_image(Image.fromarray(window))
        prediction = model.predict(image)
        display_prediction(prediction)
        cv2.imshow('Draw a Digit', np.array(window))

# Feche a janela quando o loop terminar
cv2.destroyAllWindows()
